In [1]:
import os
import numpy as np
from tensorflow.keras.models import load_model
from sklearn.model_selection import train_test_split
import tensorflow as tf
from scipy.ndimage import zoom
from skimage.metrics import structural_similarity as calculate_ssim
from PIL import Image
import matplotlib.pyplot as plt

In [2]:
lr_images_path = '../data/lr_images'
hr_images_path = '../data/hr_images'

In [3]:
lr_filenames = sorted(os.listdir(lr_images_path))[:]
hr_filenames = sorted(os.listdir(hr_images_path))[:]

# Verify that the filenames match
if lr_filenames != hr_filenames:
    raise ValueError("Filenames in lr_images and hr_images do not match!")

In [4]:
lr_images = []
hr_images = []


for img_name in lr_filenames:
    lr_img = Image.open(os.path.join(lr_images_path, img_name))
    hr_img = Image.open(os.path.join(hr_images_path, img_name))
    
    # Convert grayscale images to RGB format
    if lr_img.mode != 'RGB':
        lr_img = lr_img.convert('RGB')
        
    if hr_img.mode != 'RGB':
        hr_img = hr_img.convert('RGB')
    
    lr_img = np.asarray(lr_img)/255.0
    hr_img = np.asarray(hr_img)/255.0
    
    lr_images.append(lr_img)
    hr_images.append(hr_img)

In [5]:
lr_images_array = np.stack(lr_images)
hr_images_array = np.stack(hr_images)

print("LR Images Array Shape:", lr_images_array.shape)
print("HR Images Array Shape:", hr_images_array.shape)

LR Images Array Shape: (15129, 64, 64, 3)
HR Images Array Shape: (15129, 128, 128, 3)


In [6]:
# Split the data into training, validation, and test sets
random_seed = 42 

train_lr_images, val_test_lr_images, train_hr_images, val_test_hr_images = train_test_split(
    lr_images_array, hr_images_array, test_size=0.3, random_state=random_seed
)

val_lr_images, test_lr_images, val_hr_images, test_hr_images = train_test_split(
    val_test_lr_images, val_test_hr_images, test_size=0.5, random_state=random_seed
)

In [9]:
generator = load_model('results/generator_ckpts/gen_e_8.h5')

Metal device set to: Apple M2

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2023-09-20 11:33:24.732613: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-09-20 11:33:24.734197: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [10]:
srgan_hr_predictions = generator.predict(test_lr_images)

2023-09-20 11:33:26.070286: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-09-20 11:33:26.282852: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


71/71 [==============================] - 77s 1s/step


In [16]:
min_val = 0
max_val = 1
# Clip values to the specified range
srgan_hr_predictions = np.clip(srgan_hr_predictions, min_val, max_val)

In [41]:
# def bicubic_upscale(image, scale_factor = 2):
#     """
#     Perform bicubic interpolation for upscaling.

#     Parameters:
#         image (numpy.ndarray): Input image of shape (height, width, channels).
#         scale_factor (float): Scaling factor.

#     Returns:
#         numpy.ndarray: Upscaled image.
#     """
#     # Calculate the target height and width after upscaling
#     target_height, target_width = map(int, np.array(image.shape[:2]) * scale_factor)

#     # Perform bicubic interpolation using scipy.ndimage.zoom
#     upscaled_image = zoom(image, (scale_factor, scale_factor, 1), order=3)

#     return upscaled_image


import cv2

def bicubic_upscale(img, target_height=128, target_width=128):
    """
    Perform bicubic interpolation to upscale the input image.

    Parameters:
        img (numpy.ndarray): Input image (HxWxC).
        target_height (int): Desired height for upscaling.
        target_width (int): Desired width for upscaling.

    Returns:
        numpy.ndarray: Upscaled image (target_height x target_width x C).
    """
    # Perform bicubic interpolation
    upscaled_img = cv2.resize(img, (target_width, target_height), interpolation=cv2.INTER_CUBIC)

    return upscaled_img

def nearest_upscale(img, target_height=128, target_width=128):
    """
    Perform nearest-neighbor interpolation to upscale the input image.

    Parameters:
        img (numpy.ndarray): Input image (HxWxC).
        target_height (int): Desired height for upscaling.
        target_width (int): Desired width for upscaling.

    Returns:
        numpy.ndarray: Upscaled image (target_height x target_width x C).
    """
    # Perform nearest-neighbor interpolation
    upscaled_img = cv2.resize(img, (target_width, target_height), interpolation=cv2.INTER_NEAREST)

    return upscaled_img

In [18]:
def calculate_psnr(img1, img2):
    """
    Calculate PSNR (Peak Signal-to-Noise Ratio) for a pair of images.

    Parameters:
    img1 (numpy.ndarray): Ground truth image.
    img2 (numpy.ndarray): Predicted or reconstructed image.

    Returns:
    float: PSNR value.
    """
    mse = np.mean((img1 - img2) ** 2)
    if mse == 0:
        return float('inf')
    max_pixel = np.max(img1)
    psnr = 10 * np.log10((max_pixel ** 2) / mse)
    return psnr


In [19]:
def ssim_score(img1, img2):
    """
    Calculate SSIM (Structural Similarity Index) between two images.

    Parameters:
        img1 (numpy.ndarray): First input image (grayscale or color).
        img2 (numpy.ndarray): Second input image (grayscale or color).

    Returns:
        float: SSIM value between -1 and 1.
    """
    return calculate_ssim(img1, img2, multichannel=True)

In [21]:
srgan_psnr =[]
srgan_ssim = []
for pred, gt in zip(srgan_hr_predictions, test_hr_images):
    srgan_psnr.append(calculate_psnr(pred, gt))
    srgan_ssim.append(ssim_score(pred, gt))

srgan_psnr_score = np.mean(srgan_psnr)
srgan_ssim_score = np.mean(srgan_ssim)

/var/folders/pw/3vr89zzj1xvbzvtk1yfz9kkc0000gn/T/ipykernel_54901/2036621558.py:12: FutureWarning: `multichannel` is a deprecated argument name for `structural_similarity`. It will be removed in version 1.0. Please use `channel_axis` instead.
  return calculate_ssim(img1, img2, multichannel=True)
/Users/anwesh/opt/anaconda3/envs/tf/lib/python3.9/site-packages/skimage/_shared/utils.py:348: UserWarning: Inputs have mismatched dtype.  Setting data_range based on im1.dtype.
  return func(*args, **kwargs)


In [45]:
bicubic_interpolations = []
nearest_interpolations = []

for img in test_hr_images:
    bicubic_interpolations.append(bicubic_upscale(img))
    nearest_interpolations.append(nearest_upscale(img))


In [46]:
bicubic_psnr = []
bicubic_ssim = []

for pred, gt in zip(bicubic_interpolations, test_hr_images):
    bicubic_psnr.append(calculate_psnr(pred, gt))
    bicubic_ssim.append(ssim_score(pred, gt))

bicubic_psnr_score = np.mean(bicubic_psnr)
bicubic_ssim_score = np.mean(bicubic_ssim)

/var/folders/pw/3vr89zzj1xvbzvtk1yfz9kkc0000gn/T/ipykernel_54901/2036621558.py:12: FutureWarning: `multichannel` is a deprecated argument name for `structural_similarity`. It will be removed in version 1.0. Please use `channel_axis` instead.
  return calculate_ssim(img1, img2, multichannel=True)


In [47]:
nearest_psnr = []
nearest_ssim = []

for pred, gt in zip(nearest_interpolations, test_hr_images):
    nearest_psnr.append(calculate_psnr(pred, gt))
    nearest_ssim.append(ssim_score(pred, gt))

nearest_psnr_score = np.mean(nearest_psnr)
nearest_ssim_score = np.mean(nearest_ssim)

/var/folders/pw/3vr89zzj1xvbzvtk1yfz9kkc0000gn/T/ipykernel_54901/2036621558.py:12: FutureWarning: `multichannel` is a deprecated argument name for `structural_similarity`. It will be removed in version 1.0. Please use `channel_axis` instead.
  return calculate_ssim(img1, img2, multichannel=True)


In [48]:
print(f"SRGAN PSNR: {srgan_psnr_score}")
print(f"SRGAN SSIM SCORE: {srgan_ssim_score}")

print(f"BICUBIC INTEROLATION PSNR: {bicubic_psnr_score}")
print(f"BICUBIC INTEROLATION SSIM SCORE: {bicubic_ssim_score}")

print(f"NEAREST INTEROLATION PSNR: {nearest_psnr_score}")
print(f"NEAREST INTEROLATION SSIM SCORE: {nearest_ssim_score}")

SRGAN PSNR: 30.07404691783643
SRGAN SSIM SCORE: 0.9633726477622986
BICUBIC INTEROLATION PSNR: inf
BICUBIC INTEROLATION SSIM SCORE: 1.0
NEAREST INTEROLATION PSNR: inf
NEAREST INTEROLATION SSIM SCORE: 1.0
